# Context menus

**Context menus are small context-sensitive menus which typically appear when right clicking on an item of the QT window**. Qt has support for generating these menus, and widgets have a `specific event` used to trigger them. If you are not familiar with `signal, slot or event`. Go check the [L08_Signals_slots.ipynb](../L08_Signals_slots.ipynb)


## A simple example

In the following example we're going to intercept the `.contextMenuEvent` of the `QMainWindow`. This event is fired whenever a context menu is about to be shown, and is passed a single value event of type **QContextMenuEvent**.

To intercept the event, we simply override the method **contextMenuEvent** with our new method of the same name. So our implementation of **contextMenuEvent** will handle all events of this type.

Inside the `contextMenuEvent` method, we create a **QMenu instance** and add three actions to it (action1, action2, and action3).

When the `contextMenuEvent` is triggered, we call the **exec** method of the `QMenu instance` to display the context menu at the global position of the event. `The selected action is returned by the exec method`, and we can check which action was selected using if statements.

In [2]:
import sys

from PyQt6.QtCore import Qt
from PyQt6.QtGui import QAction
from PyQt6.QtWidgets import QApplication, QLabel, QMainWindow, QMenu, QWidget, QVBoxLayout


class MainWindow(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("My App")
        # create a widget with a vbox layout
        mainWidget = QWidget()
        layout = QVBoxLayout()
        mainWidget.setLayout(layout)
        # set the widget as central widget of the window
        self.setCentralWidget(mainWidget)

        #
        self.output = QLabel("Right-click me!",self)
        layout.addWidget(self.output)

    def contextMenuEvent(self, event):
        contextMenu = QMenu(self)
        action1 = contextMenu.addAction("Action 1")
        action2 = contextMenu.addAction("Action 2")
        action3 = contextMenu.addAction("Action 3")

        action=contextMenu.exec(event.globalPos())
        print(action.text())
        if action == action1:
            self.output.setText("Action 1 is selected")
        elif action == action2:
            self.output.setText("Action 2 is selected")
        elif action == action3:
            self.output.setText("Action 3 is selected")
        else:
            self.output.setText("Unknown action")


app = QApplication(sys.argv)

window = MainWindow()
window.show()

app.exec()

Action 1
Action 2
Action 3


0

If you run the above code and right-click within the window, you'll see a context menu appear. You can set up .triggered slots on your menu actions as normal (and re-use actions defined for menus and toolbars).

## A more realistic example

In this example, we create a QMainWindow and a QTableView. When we right click on a cell of the table view, a contextMenu will be shown. If user select an action, a corresponding action will be executed.

In [ ]:
from PyQt6.QtCore import QAbstractTableModel
from PyQt6.QtWidgets import QApplication, QMainWindow, QTableView, QMenu, QLabel, QWidget, QVBoxLayout

class MyTableModel(QAbstractTableModel):
    def __init__(self, data):
        super().__init__()
        self.data = data

    def rowCount(self, parent=None):
        return len(self.data)

    def columnCount(self, parent=None):
        return len(self.data[0])

    def data(self, index, role=0):
        """
        this function is called when the table view populate the widget
        Parameters
        ----------
        index :
        role :

        Returns
        -------

        """
        if role == 0:
            return str(self.data[index.row()][index.column()])

    def getValue(self, index):
        """
        This function is called when we want to get the cell value with an given index
        Parameters
        ----------
        index :

        Returns
        -------

        """
        return self.data[index.row()][index.column()]

class MyTableView(QTableView):
    def __init__(self, data):
        super().__init__()
        self.setModel(MyTableModel(data))

    def contextMenuEvent(self, event):
        # Get the index of the clicked item
        selectedIndex = self.indexAt(event.pos())

        if selectedIndex.isValid():
            contextMenu = QMenu(self)

            # Create actions for the context menu
            action1 = contextMenu.addAction("Action 1")
            action2 = contextMenu.addAction("Action 2")

            # Connect the actions to your desired slots or functions
            action1.triggered.connect(lambda: self.onAction1Triggered(selectedIndex))
            action2.triggered.connect(lambda: self.onAction2Triggered(selectedIndex))


            # Show the context menu at the event's global position
            contextMenu.exec(event.globalPos())

    def onAction1Triggered(self, index):
        # Handle Action 1 for the clicked item
        itemValue = self.model().getValue(index)
        print(f"Action 1 triggered for item at: {index.row()} {index.column()} with value {itemValue}")

    def onAction2Triggered(self, index):
        # Handle Action 2 for the clicked item
        itemValue = self.model().getValue(index)
        print(f"Action 2 triggered for item at: {index.row()} {index.column()} with value {itemValue}")

class MainWindow(QMainWindow):
    LABEL_TXT= "The selected data cell:"
    def __init__(self):
        super().__init__()
        self.setWindowTitle("My App")

        self.data = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

        # create a table view
        self.tableView = MyTableView(self.data)
        self.setCentralWidget(self.tableView)
        # second item is a qlabel
        self.output = QLabel(MainWindow.LABEL_TXT)

        # create a widget with a vbox layout
        mainWidget = QWidget()
        layout = QVBoxLayout()

        # add table
        layout.addWidget(self.tableView)
        # add second button
        layout.addWidget(self.output)

        mainWidget.setLayout(layout)
        # set the widget as central widget of the window
        self.setCentralWidget(mainWidget)

    def handleCustomSignal(self, index):
        self.output.setText(f"{MainWindow.LABEL_TXT} rowNumber: {index.row()}, columnNumber: {index.column()}")

app = QApplication([])
window = MainWindow()


window.show()
app.exec()


qt.qpa.xcb: X server does not support XInput 2


0 0
Action 2 triggered for item at: 0 0 with value 1
0 1
Action 1 triggered for item at: 0 1 with value 2
2 1
Action 2 triggered for item at: 2 1 with value 8
